In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd
import pickle


In [ ]:
data_path = None

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
data_path

In [ ]:
cbr_datadir = os.path.join(data_path, 'Canberra')
cbr_datadir_out = os.path.join(cbr_datadir, 'out')
ngis_datadir = os.path.join(data_path, 'NGIS')
act_shp_datadir = os.path.join(ngis_datadir, 'shp_ACT')
bidgee_shp_datadir = os.path.join(ngis_datadir, 'shp_murrumbidgee_river')

In [ ]:
classified_logs_filename = os.path.join(cbr_datadir_out,'classified_logs.pkl')
with open(classified_logs_filename, 'rb') as handle:
    df = pickle.load(handle)

In [ ]:
geoloc_filename = os.path.join(cbr_datadir_out,'geoloc.pkl')
with open(geoloc_filename, 'rb') as handle:
    geoloc = pickle.load(handle)
    

In [ ]:
DEPTH_FROM_COL = 'FromDepth'
DEPTH_TO_COL = 'ToDepth'

TOP_ELEV_COL = 'TopElev'
BOTTOM_ELEV_COL = 'BottomElev'

LITHO_DESC_COL = 'Description'
HYDRO_CODE_COL = 'HydroCode'

HYDRO_ID_COL = 'HydroID'
BORE_ID_COL = 'BoreID'

In [ ]:
keep = set(df[HYDRO_ID_COL].values)
s = geoloc[HYDRO_ID_COL]

In [ ]:
geoloc = geoloc[s.isin(keep)]

# Viewer

Derived from [voila-gpx-viewer](https://github.com/jtpio/voila-gpx-viewer) and [ipyleaflet docs](https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html)

However this is very difficult to adapt, actually use case is quite different from the GPX data. ipyleaflet doc is sparse and how to I guess the content of callbacks??
https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6




In [ ]:
# from bqplot import Axis, Figure, Lines, LinearScale
# from bqplot.interacts import IndexSelector
# from ipyleaflet import basemaps, FullScreenControl, LayerGroup, Map, MeasureControl, Polyline, Marker, MarkerCluster, CircleMarker, WidgetControl
# from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link
from ipywidgets import Output
from ipyleaflet import Map, Marker, MarkerCluster, basemaps

In [ ]:
out = Output(layout={'border': '1px solid black'})

In [ ]:
class GlobalThing:
    def __init__(self):
        self.marker_info = dict()
    
    def add_marker_info(self, lat, lon, code):
        self.marker_info[(lat, lon)] = code
    
    def get_code(self, lat, lon):
        return self.marker_info[(lat, lon)]
    
globalthing = GlobalThing()

In [ ]:
def data_for_hydroid(ident):
    df_sub = df.loc[df['HydroID'] == ident]
    return df_sub[['BoreID', 'FromDepth', 'ToDepth', 'MajorLithCode',
       'MinorLithCode', 'Description', 
       'Lithology_1', 'Lithology_2']]

In [ ]:
# data_for_hydroid(80000628)

In [ ]:
def click_handler(**kwargs):
    blah = dict(**kwargs)
    xy = blah['coordinates']
    ident = globalthing.get_code(xy[0], xy[1])
    bore_data = data_for_hydroid(ident)
    out.clear_output()
    with out:
        print(ident)        
        print(bore_data)

In [ ]:
def plot_map(geoloc):
    """
    Plot the GPS trace on a map
    """
    mean_lat = geoloc.Latitude.mean()
    mean_lng = geoloc.Longitude.mean()
    # create the map
    m = Map(center=(mean_lat, mean_lng), zoom=12, basemap=basemaps.Stamen.Terrain)
    # show trace
    markers = []
    for index, row in geoloc.iterrows():
        #message = HTML()
        #message.value = row.HydroCode
        #message.placeholder = "Some HTML"
        #message.description = "Some HTML"
        globalthing.add_marker_info(row.Latitude, row.Longitude, row.HydroID)
        marker = Marker(location=(row.Latitude, row.Longitude))
        marker.on_click(click_handler)
        #marker.popup = message
        markers.append(marker)
    marker_cluster = MarkerCluster(
        markers=markers
    )
    #marker_cluster.on_click(click_handler)
    m.add_layer(marker_cluster);
    # m.add_control(FullScreenControl())
    return m

In [ ]:
plot_map(geoloc)

In [ ]:
out

## Experiment

In [ ]:
def plot_borehole_data(df):
    # We may have something fancy visual down the track, for now, a dataframe subsetting. 
    px = [p.Time for p in points]
    py = [p.CND_011 for p in points]

    x_scale, y_scale = LinearScale(), LogScale()
    x_scale.allow_padding = False
    x_ax = Axis(label='Time (s)', scale=x_scale)
    y_ax = Axis(label='CND 011(?)', scale=y_scale, orientation='vertical')

    lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})

    elevation = Figure(title='CND 011 Chart', axes=[x_ax, y_ax], marks=[lines])
    elevation.layout.width = 'auto'
    elevation.layout.height = 'auto'
    elevation.layout.min_height = '500px'

    elevation.interaction = IndexSelector(scale=x_scale)

    return elevation

In [ ]:
def link_geo_borehole(geomap, boreholelayer):
    """
    Links the geolocation of the markers to the display of the bvorehole log
    Changing the selection on the marker will update the
    borehole display
    """
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    geomap.add_control(autocenter_control)

    brushintsel = geomap.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')

    
def link_trace_elevation(trace, elevation, points):
    """
    Link the trace the elevation graph.
    Changing the selection on the elevation will update the
    marker on the map
    """
    times = np.asarray([p.Time for p in points])

    def find_point(time):
        """
        Find a point given the time
        """
        dist_1 = abs(times - time)
        pos = np.argmin(dist_1)
        return points[pos]
    
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    trace.add_control(autocenter_control)
    # mark the current position on the map
    start = points[0]
    marker = CircleMarker(visible=False, location=(start.Latitude, start.Longitude),
                          radius=10, color="green", fill_color="green")
    trace.add_layer(marker)
    brushintsel = elevation.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')


In [ ]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

In [ ]:
def plot_gpx(points):
    trace = plot_map(points)
    elevation = plot_elevation(points)
    debug = Label(value='')
    display(trace)
    display(elevation)
    display(debug)
    link_trace_elevation(trace, elevation, points)

In [ ]:
plot_gpx(points)